In [109]:
import pandas as pd
import numpy as np 
import seaborn as sns
import datetime
import calendar
import matplotlib.pyplot  as plt

pd.set_option('display.max_columns', None)

#### Reading Dataset

In [110]:
data_before = pd.read_csv("Collisions.csv")

## Taking a glimpse of our dataset

In [111]:
data_before.head(2)

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,-122.386772,47.564720,1,326234,327734,E984735,Matched,Intersection,31893.0,CALIFORNIA AVE SW AND SW GENESEE ST,,NaN,2,Injury Collision,Pedestrian,2,1,0,1,1,0,0,2019/11/20 00:00:00+00,11/20/2019 10:25:00 AM,At Intersection (intersection related),24.0,MOTOR VEHCILE STRUCK PEDESTRIAN,NaN,N,Clear,Dry,Daylight,Y,NaN,NaN,2,Vehicle turning left hits pedestrian,0,0,N
1,-122.341806,47.686934,2,326246,327746,E985430,Matched,Intersection,24228.0,STONE AVE N AND N 80TH ST,,NaN,2,Injury Collision,Angles,4,0,0,2,2,0,0,2019/11/18 00:00:00+00,11/18/2019 5:01:00 PM,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Raining,Wet,Dark - Street Lights On,NaN,NaN,NaN,10,Entering at angle,0,0,N


In [112]:
data_before.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221266 entries, 0 to 221265
Data columns (total 40 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   X                213797 non-null  float64
 1   Y                213797 non-null  float64
 2   OBJECTID         221266 non-null  int64  
 3   INCKEY           221266 non-null  int64  
 4   COLDETKEY        221266 non-null  int64  
 5   REPORTNO         221266 non-null  object 
 6   STATUS           221266 non-null  object 
 7   ADDRTYPE         217554 non-null  object 
 8   INTKEY           71823 non-null   float64
 9   LOCATION         216680 non-null  object 
 10  EXCEPTRSNCODE    100863 non-null  object 
 11  EXCEPTRSNDESC    11775 non-null   object 
 12  SEVERITYCODE     221265 non-null  object 
 13  SEVERITYDESC     221266 non-null  object 
 14  COLLISIONTYPE    194767 non-null  object 
 15  PERSONCOUNT      221266 non-null  int64  
 16  PEDCOUNT         221266 non-null  int6

#### Excluding columns that can't be considered as a feature.
1. OBEJECTID
2. SHAPE
3. INCKEY
4. COLDETKEY
5. ADDRTYPE
6. INTKEY
7. EXCEPTRSNCODE
8. EXCEPTRSNDESC
9. SEVERITYDESC
10. SDOTCOLNUM
11. ST_COLDESC
12. X
13. Y
14. REPORTNO
15. STATUS
16. LOCATION
17. INJURIES
18. SERIOUSINJURIES
19. FATALITIES
20. SEGLANEKEY
21. CROSSWALKKEY


#### Considering columns that will take part in model building
1.  ADDRTYPE
2.  SEVERITYCODE
3.  COLLISIONTYPE
4. PERSONCOUNT
5.  PEDCOUNT
6.  PEDCYLCOUNT
7.  VEHCOUNT
8.  INCDATE
9.  INCDTTM
10. JUNCTIONTYPE
12. UNDERINFL
13. WEATHER
14. ROADCOND
15. LIGHTCOND
16. PEDROWNOTGRNT
17. SPEEDING
18. ST_COLCODE
19. HITPARKEDCAR

In [113]:
#  Making a list of columns which we will remove

remove_cols = ['OBJECTID','INCKEY','COLDETKEY','INTKEY','EXCEPTRSNCODE','EXCEPTRSNDESC','SEVERITYDESC','SDOTCOLNUM','ST_COLDESC','X','Y','REPORTNO','STATUS','LOCATION','SDOT_COLDESC','SDOT_COLCODE','INJURIES','SERIOUSINJURIES','FATALITIES','SEGLANEKEY','CROSSWALKKEY','INATTENTIONIND','INCDATE','ST_COLCODE','PEDCOUNT','PERSONCOUNT','PEDCYLCOUNT','VEHCOUNT']

In [114]:
# Removing unusual columns from dataset

data_after = data_before.drop(columns=remove_cols, axis=1)

In [115]:
data_before.shape

(221266, 40)

In [116]:
data_after.shape

(221266, 12)

In [117]:
data_after['SEVERITYCODE'].value_counts()

1     137485
2      58698
0      21635
2b      3098
3        349
Name: SEVERITYCODE, dtype: int64

In [118]:
# Removing rows having Severitycode - 0
data_after['SEVERITYCODE'].replace('0',np.nan, inplace=True)

In [119]:
data_after.dropna(axis=0, subset=['SEVERITYCODE'], inplace=True)

In [120]:
# Replacing 2b and 3 with 3 and 4 for ease to use

data_after['SEVERITYCODE'].replace('3', '4',inplace=True)
data_after['SEVERITYCODE'].replace('2b', '3',inplace=True)

In [121]:
data_after['SEVERITYCODE'].value_counts()

1    137485
2     58698
3      3098
4       349
Name: SEVERITYCODE, dtype: int64

#### As we can see, This data is highly unbalanced and have more occurences around the Severtiy code '1'.
#### We have to balance it using resampling techniques else the model will give highly biased result favoring severitycode '1'.

#### But before, we need to select features for our models.

In [122]:
data_after.isnull().sum()

ADDRTYPE           1931
SEVERITYCODE          0
COLLISIONTYPE      4865
INCDTTM               0
JUNCTIONTYPE       6375
UNDERINFL          4845
WEATHER            5055
ROADCOND           4975
LIGHTCOND          5143
PEDROWNOTGRNT    194443
SPEEDING         189718
HITPARKEDCAR          0
dtype: int64

In [123]:
data_after['UNDERINFL'].value_counts()

N    103480
0     81676
Y      5399
1      4230
Name: UNDERINFL, dtype: int64

#### Above we can see, 'N' and '0' refers same so as 'Y' and '1'.

In [124]:
data_after['UNDERINFL'].replace('N','0',inplace=True)
data_after['UNDERINFL'].replace('Y','1',inplace=True)

In [125]:
data_after['UNDERINFL'].value_counts()

0    185156
1      9629
Name: UNDERINFL, dtype: int64

#### We can most of the columns are categorical and are having null values.

##### We will be filling these values with equal distribution of the categories in each column

## Speeding feature only has 'Y' and rest are NaN

In [126]:
data_after['SPEEDING'].value_counts()

Y    9912
Name: SPEEDING, dtype: int64

In [127]:
data_after['SPEEDING'].isnull().sum()

189718

In [128]:
data_after['SPEEDING'].replace('Y',int(1),inplace=True)
data_after['SPEEDING'].replace(np.nan,int(0),inplace=True)


In [129]:
data_after['SPEEDING'].astype(int)

0         0
1         0
2         0
3         0
6         0
         ..
221261    0
221262    0
221263    0
221264    0
221265    0
Name: SPEEDING, Length: 199630, dtype: int64

In [130]:
data_after['SPEEDING'].tail(500)

220679    0.0
220680    0.0
220681    0.0
220682    0.0
220685    0.0
         ... 
221261    0.0
221262    0.0
221263    0.0
221264    0.0
221265    0.0
Name: SPEEDING, Length: 500, dtype: float64

In [131]:
def fill_nulls(dataframe, cols):
    df = dataframe
    for c in cols:
        s = df[c].value_counts(normalize=True)
        missing = df[c].isnull()
        df.loc[missing,c] = np.random.choice(s.index, size=len(df[missing]),p=s.values)

In [132]:
fill_null_cols = ['ADDRTYPE', 'COLLISIONTYPE','JUNCTIONTYPE','UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT']

In [133]:
fill_nulls(data_after,fill_null_cols)


In [134]:
data_after.isnull().sum()


ADDRTYPE         0
SEVERITYCODE     0
COLLISIONTYPE    0
INCDTTM          0
JUNCTIONTYPE     0
UNDERINFL        0
WEATHER          0
ROADCOND         0
LIGHTCOND        0
PEDROWNOTGRNT    0
SPEEDING         0
HITPARKEDCAR     0
dtype: int64

In [135]:
data_after['SPEEDING'] = data_after['SPEEDING'].astype(int)

In [136]:
# Extracting year, month,day to get insights on individual

data_after['year'] = pd.DatetimeIndex(data_after['INCDTTM']).year
data_after['month'] = pd.DatetimeIndex(data_after['INCDTTM']).month
data_after['day'] = pd.DatetimeIndex(data_after['INCDTTM']).day

data_after['month'] = data_after['month'].apply(lambda x: calendar.month_abbr[x])

In [137]:
data_after.head(1)

,ADDRTYPE,SEVERITYCODE,COLLISIONTYPE,INCDTTM,JUNCTIONTYPE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,HITPARKEDCAR,year,month,day
0,Intersection,2,Pedestrian,11/20/2019 10:25:00 AM,At Intersection (intersection related),0,Clear,Dry,Daylight,Y,0,N,2019,Nov,20


## We have performed cleaning data, Now we can save this and can use for exploratory data analysis.

In [138]:
data_after.to_csv("data_cleaned.csv",index=False)